[**<< Back to first page**](./index.ipynb)

## Data generating process

Let $m_{i,t}=[m_1,m_2]^T_{i,t}$ be the $2 \times 1$ vector corresponding to the mood values from the $i$th day at any observation occasion $t=1,2,\ldots,n_i$ where $n_i$ is the number of observations per day and $T$ denotes the transpose of a matrix.

Assuming that a collection of external factors (E) influence moods and their effect could vary between days, the evolution of $m_{i,t}$ is represented by a LARMEx model as
$$
m_{i,t} = (\beta^{ar}+b^{ar}_i)m_{i,t-1} + (\beta^e+b^e_i)e_{i,t} + (\beta^c+b^c_i) + \epsilon_{i,t},
$$
in which $\beta$ and $b$ represent the fixed and random effects (FE and RE henceforth). The temporal (Granger-causal) connections
between the moods are governed by the lag(1) autoregressive term, $(\beta^{ar}+b^{ar}_i)m_{i,t-1}$. The remaining terms represent the contemporaneous effects of E and the constants (intercepts). For simplicity, we assume no connections from moods to E. 

## Generating a set of known parameters

For simulating data with this model, we need to specify the parameters. To achieve 
replicability, one can predefine a random number generator (RNG). In Julia, we 
implement this using the following `struct`.
```Julia
mutable struct parLARMEx
    nAR::Int              # number of temporally connected nodes
    rng::AbstractRNG      # RNG throught the simulation for consintency
    nL2Max::Int           # how many random-effects to generate
    nSamp::Int            # sample size for generating REs, see genRE_CS()
    B_AR::Matrix{Float64} # fixed-effects autoregressive coefficients
    B_E::Vector{Float64}  # fixed-effects coefficients of exogenous factors
    b_var::Matrix{Float64}# variance of group of random-effects
    b_cov::Matrix{Float64}# variance-covariance matrix of random-effects
    b_ar::Matrix{Float64} # random-effects for autoregressive coefficients
    b_e::Matrix{Float64}  # random-effects for exogenous coefficients
    b_c::Matrix{Float64}  # random-effects for constant terms
end
```

To generate instances of this `struct`, a constructor is implemented as
```Julia
parLARMEx(;b=[], nVar=2, rng=MersenneTwister(), nL2Max=100,  
           nSamp=20000, B_ar=.3, B_e=.3, b_var=[.03 .03 .03]) 
```
where the keyword arguments are:

- `b = []`: if provided with a matrix, RE are extracted from it otherwise generated
- `nVar = 2`: number of temporally connected nodes  
- `rng = MersseneTwister()`: left out, a `MersenneTwister` RNG
	is produced anew, otherwise a custom one should be provide for consistency and 
	replication
- `nL2Max = 100`: how many RE to generate
- `nSamp = 20000`: initial sample size for generating RE, see genRE_CS() for explanation
- `B_ar = .3`: absolute value of FE autoregressive coefficients, `= []` for random values between 0.1 and 0.6
- `B_e = .3`: absolute value of FE exogenous coefficients, `= []` for random values between 0.1 and 0.6
- `b_var = [.03 .03 .03]`: for constructing a default variance-covariance of RE 

For simplicity we restrict the fixed-effects to be

$$
\beta^{ar}=\begin{bmatrix}0.3 & -0.3\\-0.3 &  0.3\end{bmatrix}, \quad 
\beta^{e}=\begin{bmatrix} 0.3\\ -0.3 \end{bmatrix}, \quad 
\beta^{c}=\begin{bmatrix} 0\\ 0 \end{bmatrix} 
$$

These matrices  are generated using keyword arguments to the constructor of `parLARMEx`. It is also possible to initialize these with random values by passing `[]`. 
The variance-covariance of the random-effects is built using `b_var` to be


\begin{equation*}
G =  \begin{bmatrix}
0.03&0&0&0&0&0&0&0\\
0&0.03&0&0&0&0&0&0\\
0&0&0.03&0&0&0&0&0\\
0&0&0&0.03&0&0&0&0\\
0&0&0&0&0.03&0&0&0\\
0&0&0&0&0&0.03&0&0\\
0&0&0&0&0&0&0.03&0\\
0&0&0&0&0&0&0&0.03 
\end{bmatrix}
\end{equation*}

This is not the final variance-covariance of the RE because one needs to control for the stability of the autoregressive process. To this end, we sample a numger of `nSamp = 20000` sets from a multivariate normal distribution of $ MVN(0,G) $ and retain `nL2Max = 100` sets for which the absoloute eigenvalues of $ \beta^{ar}+b^{ar} $ are less than one. These are stored in `b_ar`, `b_e`, `b_c`, and the variance-covariance of this latter sample as `b_cov`. 

This process is accomplished by calling the function 
```Julia
genRE_CS(rng,G,B_AR,maxSbj,nSamp) 
```
inside the constructor which updates the fields corresponding to the random-effects and their variance-covariance structure. In what follows we import the function definitions saved at the file `helperSim.jl`, included in the appendix, and then construct the parameters using a replicable RNG.

In [1]:
include("helperSim.jl");

In [2]:
rng = MersenneTwister(1984)
par = parLARMEx(rng=rng);

## Generating data

We assume that the data generating process has a two-level structure. The multiple observations are made on level I and these are nested in level II units. For EMA data, level I could be the daily observations, as many as `nObsL1`, which are nested in single days with a total number of `nL2`. These characteristics together with the varianc-covariance of noise, `sigma`, initial values for days, `S0`, and the vector of known exogenous factors, `E`, are stored in another `struct`. 
```Julia
mutable struct simLARMEx
    nAR::Int              # number of temporally connected nodes
    nObsL1::Int           # number of observation on level I            
    nL2::Int              # number of units on level II
    sigma::Float64        # variance of noise
    M0::Matrix{Float64}   # initial values each day
    E::Vector{Float64}    # known exogenous factors
end
```
The constructor of this `struct` has the following signature,
```Julia
simLARMEx(;rng=MersenneTwister(), nObsL1=10, nL2=36, sigma=.2, M0_max=.5, E=[])
```
and keyword arguments as: 

- `rng = MersenneTwister()`: feed `parLARMEx.rng` for consistency and replication, initialized anew by default
- `nObsL1 = 10`: number of observation on level I 
- `nL2 = 36`: number of units on level II
- `sigma = .02`: variance of noise 
- `M0_max = .5`: determines the amplitude of initial values and exogenous factors, 0.5 to keep trajectories mostly in [-1,1]
- `E = []`: known exogenous factors of size [`nL2` x `nObsL1`], drawn randomly from [`0`, `M0_max`] by default

The following code snippet setts up the configuration of the desired data set. The RNG is carried over to perovide replicablity. Finally, `genData()` gives the simulated data along with the noiseless data as dataframes and the signal-to-noise-ratio (SNR). 

In [3]:
sim = simLARMEx(rng=par.rng);
simData, simData0, SNR = genData(par, sim);

In [4]:
show(first(simData,5),allcols=true)
# latexify(round.(simData[1:5,:],digits=2))

5×5 DataFrame
 Row │ idL2   tL1    M1          M2         E        
     │ Int64  Int64  Float64     Float64    Float64  
─────┼───────────────────────────────────────────────
   1 │     1      1  -0.0714286   0.438776  0.265306
   2 │     1      2  -0.0271097   0.1977    0.489796
   3 │     1      3   0.204667    0.083435  0.193878
   4 │     1      4   0.30021    -0.039331  0.234694
   5 │     1      5   0.193203   -0.165018  0.255102

The returned values are

- `simData`: the simulated data according to the setup so far
- `simData0`: the simulated data without the noise component used to calculate the SNR
- `SNR`: the ratio of the variance of noiseless data to that of noise

## Transforming to classical mixed-effects model

By stacking mood values, parameters and covariates for every day separately, and forming the design matrices
for fixed and random effects, $X$ and $Z$, data from a single day takes the following matrix form which is 
equivalent to a linear mixed effects formulation.
$$
Y_i = X_i\beta + Z_i b_i + \epsilon_i
$$

In this formulation for a network of two moods and one external nodes one has

$$
Y_i = \begin{bmatrix}
m_{1,1} & m_{1,2} & \ldots & m_{1,n_i} & m_{2,1} & m_{2,2} & \ldots & m_{2,n_i}
\end{bmatrix}^T_i,
$$
$$
\beta = \begin{bmatrix}
\beta_{11}   & \beta_{12}   & \beta_{21}   & \beta_{22} & \beta^{e}_1 &
\beta^{e}_2 &   \beta^{c}_1 & \beta^{c}_2
\end{bmatrix}^T,
$$
and

$$
X_i = Z_i = \begin{bmatrix}
m_{1,0}     & m_{2,0}     & 0           & 0           & 1 & e_1     & 0 & 0 \\
m_{1,1}     & m_{2,1}     & 0           & 0           & 1 & e_2     & 0 & 0 \\
\vdots      & \vdots      &             &             &   & \vdots  &   &   \\
m_{1,n_i-1} & m_{2,n_i-1} & 0           & 0           & 1 & e_{n_i} & 0 & 0 \\
0           & 0           & m_{1,0}     & m_{2,0}     & 0 & 0       & 1 & e_1 \\
0           & 0           & m_{1,1}     & m_{2,1}     & 0 & 0       & 1 & e_2\\
\vdots      & \vdots      & \vdots      & \vdots      &   &         &   & \vdots\\
0           & 0           & m_{1,n_i-1} & m_{2,n_i-1} & 0 & 0       & 1 & e_{n_i}
\end{bmatrix}
$$

In its general form, the above equation for $k$ moods, $Y_i$ is a $k(n_i-1) \times 1$
vector of mood values, $\beta$ is a $(k^2+2k) \times 1$ vector of fixed effects, $b_i$ is a
$(k^2+2k) \times 1$ vector of random effects, $X_i$ and $Z_i$ are $k(n_i-1) \times (k^2+2k)$
design matrices. The random effects $b_i$ and  residuals $\epsilon_i$ are assumed to be independent with
a multivariate normal (MVN) distribution of 
$$
\begin{bmatrix} b_i \\ \epsilon_i \end{bmatrix}  \sim MVN\left(
\begin{bmatrix} {0} \\ {0} \end{bmatrix},
\begin{bmatrix} G & {0} \\ {0} & \Sigma_i \end{bmatrix}\right).
$$  

This step is done by the following function which gives another dataframe suitable for performing the estimation. Here, it is possible to introduce missing values to the analysis, e.g. `miss = .2` for a 20% of missing values at random. By default, it is assumed that there is no missing values, `miss = 0`.
```Julia
prepData2Fit(rawData, idL2, endList, exgList; miss=0)
```
with the following arguments:

- `rawData`: the simulated data as a dataframe
- `idL2`: the column name for level II units, e.g., an id for each day, `"idL2"` here
- `endList`: the list of temporelly connected moods, `["M1","M2"]` here
- `exgList`: the list of exogenous factors together with the constant terms, `["E","C"]` here 
- `miss`: the ratio of missing values, defaults to zero

In [6]:
fitData = prepData2Fit(simData, rng, "idL2", ["M1","M2"], ["E","C"]);
show(first(fitData,5),allcols=true)

5×11 DataFrame
 Row │ idL2  tL1    M           M11         M12        M21       M22       E1        E2        C1        C2       
     │ Cat…  Int16  Float64?    Float64?    Float64?   Float64?  Float64?  Float64?  Float64?  Float64?  Float64? 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 1         2  -0.0271097  -0.0714286   0.438776      -0.0       0.0  0.489796       0.0       1.0       0.0
   2 │ 1         3   0.204667   -0.0271097   0.1977        -0.0       0.0  0.193878       0.0       1.0       0.0
   3 │ 1         4   0.30021     0.204667    0.083435       0.0       0.0  0.234694       0.0       1.0       0.0
   4 │ 1         5   0.193203    0.30021    -0.039331       0.0      -0.0  0.255102       0.0       1.0       0.0
   5 │ 1         6   0.649712    0.193203   -0.165018       0.0      -0.0  0.489796       0.0       1.0       0.0

In [8]:
# df = fitData[1:5,:]; df[:,3:end] = round.(df[:,3:end],digits=2);
# latexify(df)

## Setting up the formula for fitting

The prepared data has more columns representing the response variable `M`, and the predictors including the connections in the network `M11, M12, M21, M22, E1, E2`, as well as the constant terms `C1, C2`. It also contains the level I time points `tL1`, and the level II identifiers `idL2`. We provide a function
```Julia
setFormula(fitData)
```
which constructs the formula suitable to feed in the Julia package [MixedModels.jl](https://juliastats.org/MixedModels.jl/stable/). This function is restricted only to a dataframe which has the exact columns as shown above.

In [7]:
frm = setFormula(fitData)

FormulaTerm
Response:
  M(unknown)
Predictors:
  0
  M11(unknown)
  M12(unknown)
  M21(unknown)
  M22(unknown)
  E1(unknown)
  E2(unknown)
  (M11,M12,M21,M22,E1,E2,C1,C2,idL2)->(0 + M11 + M12 + M21 + M22 + E1 + E2 + C1 + C2) | idL2

## Fitting

We use the [MixedModels.jl](https://juliastats.org/MixedModels.jl/stable/) package in Julia to perform the estimation. It is similar to the [lme4](https://github.com/lme4/lme4) in R but exhibits faster performance in our case. 

In [8]:
fit = MixedModels.fit(MixedModel, frm, fitData, REML=true, progress=false);
show(fit)

Linear mixed model fit by REML
 

M ~ 0 + M11 + M12 + M21 + M22 + E1 + E2 + (0 + M11 + M12 + M21 + M22 + E1 + E2 + C1 + C2 | idL2)
 REML criterion at convergence: -468.8576565442459



Variance components:
         ColumnVariance Std.Dev.   Corr.
idL2     M11  0.019478 0.139563
         M12  0.069804 0.264205 +0.25
         M21  0.028548 0.168961 -0.47 +0.60
         M22  0.051129 0.226118 -0.72 +0.03 +0.25
         E1   0.028903 0.170009 +0.48 +0.26 -0.10 -0.56
         E2   0.022133 0.148771 +0.77 +0.56 -0.04 -0.44 +0.26
         C1   0.032112 0.179197 -0.12 +0.20 +0.36 -0.16 +0.38 -0.45
         C2   0.031712 0.178080 +0.30 +0.05 -0.18 -0.12 -0.33 -0.04 +0.34
Residual      0.017535 0.132421
 Number of obs: 648; levels of grouping factors: 

36

  Fixed-effects parameters:


───────────────────────────────────────────
         Coef.  Std. Error      z  Pr(>|z|)
───────────────────────────────────────────


M11   0.159073   0.0445327   3.57    0.0004
M12  -0.48537    0.0606019  -8.01    <1e-14
M21  -0.28228    0.0465415  -6.07    <1e-08
M22   0.332441   0.0565884   5.87    <1e-08
E1    0.393878   0.0592017   6.65    <1e-10
E2   -0.254242   0.0544601  -4.67    <1e-05
───────────────────────────────────────────

The following parameters have been estimated. It is evident that the network's structure, 
with regard to the type of connections, has been accurately reconstructed. However, the 
precision of the coefficient estimations is not optimal.
$$
\hat{\beta}^{ar}=\begin{bmatrix}0.16 & -0.48\\-0.28 &  0.34\end{bmatrix}, \quad 
\hat{\beta}^{e}=\begin{bmatrix} 0.39\\ -0.25 \end{bmatrix}, \quad 
$$

## Parameter recovery

In order to assess how well one can recover the parameters, we generate data with the following specifications:

- number of observations per day: $10$
- $(\sigma^2,\text{SNR})\in\{(0.01,12),(0.02,6),(0.06,2)\}$
- number of days, $N \in \{4,6,\ldots,36\}$
- $\beta^{ar}$, $\beta^{e}$ and $\beta^{c}$ as before
- data is generated for one simulated subject 1000 times
- at each iteration new random effects are used

This can be done using the script `simBootstrap.jl`. It uses the followint function to generate data in a loop. 
```Julia
loopSim(csvDir, M0_max, sigma, n, P)
```
The function creates a folder named `csvDir` and for each noise intensity, `sigma`, saves the true and estimated parameters of each simulation in different folders, `re` and `reh`, based on the number of days as `CSV` files. Fixed effects and the variances of random effects are collected in two separate `CSV` files.

The code could take some time, depending on the computational power. Using the estimated parameters we construct bootstrap confidence intervals for the fixed effects and the variance of random effects. 

\includegraphics[width=\textwidth,center]{feCI} % used for the latex output of this document

<div>
<center><img src="./img/feCI.svg" width="800"/></center>
</div>

The number of observations per day is $10$. True values of parameters are highlighted bold on x axes. For every number of days at the horizontal axes, three lines are drawn representing the bootstrap $95\%$ confidence intervals around the median depicted by cross signs. Every line is color-coded to demonstrate one noise intensity. Data has been generated for one simulated subject, 1000 times repeatedly, with $(\sigma^2,\text{SNR})\in\{(0.01,12),(0.02,6),(0.06,2)\}$.

\includegraphics[width=\textwidth,center]{sigCI} % used for the latex output of this document

<div>
<center><img src="./img/sigCI.svg" width="800"/></center>
</div>

True values of parameters are highlighted bold on x axes, $\sqrt{0.03}\approx 0.17$.

For further analyses please refer to our manuscript.

## Conclusion

We presented an extension of linear mixed-effects models by adding an 
autoregressive component to model the network representation of mental 
state. Specifically, we assumed a simple network of two causally 
interacting moods under the unidirectional influence of an external-factors 
node. This framework is suitable for intensive longitudinal data. 

We showed briefly how this representation is mathematically formulated and 
detailed the implementation of such a model in Julia. Please refer to `demo.jl` 
for a clean and detailed implementation a simple estimation. We also provided 
exemplary code for performing a bootstrap analysis to construct confidence 
intervals for the estimations. The goal here was to calibrate the model through 
simulations by quantifying the estimation errors. Further work is needed to 
perform the calibration and validation using real data.  

In this text, we mainly discussed the number of observations per subject 
by adding more days for a specific intensity of noise in data. However, 
it would be straightforward to modify the provided code to simulate 
data for other real-world circumstances.